In [39]:
# Import libraries
from mido import MidiFile, MidiTrack

# Import file stuff
import os

# Import basic
import pandas as pd

In [4]:
def process_midi_file(filepath):
    midi = MidiFile(filepath)

    notes = []
    velocities = []
    durations = []

    for track in midi.tracks:
        time = 0
        for msg in track:
            time += msg.time

            if msg.type == 'note_on':
                notes.append(msg.note)
                velocities.append(msg.velocity)
                durations.append(time)
                time = 0
            elif msg.type == 'note_off':
                time += msg.time

    return notes, velocities, durations

In [6]:
filepath = './Composer_Dataset/NN_midi_files_extended/dev/bach/bach344.mid'
notes, velocities, durations = process_midi_file(filepath)

In [76]:
def get_children(a_dir):
    dirs = []
    files = []
    for name in os.listdir(a_dir):
        if os.path.isdir(os.path.join(a_dir, name)):
            dirs.append(name)
        else:
            files.append(name)
    return [dirs,files]

def create_files_table(top_level, out_file):
    temp_comps = []
    temp_songs = []
    temp_paths = []
    
    composer_names, songs = get_children(top_level)

    for composer in composer_names:
        temp_path = top_level + '/' + composer
        temp, songs = get_children(temp_path)
        for song in songs:
            if song != '.DS_Store':
                temp_comps.append(composer)
                temp_paths.append(temp_path + '/' + song)
                temp_songs.append(song.split(".")[0])

    temp_dict = {'Composers': temp_comps, 'Songs': temp_songs, 'Paths': temp_paths}

    table = pd.DataFrame.from_dict(temp_dict)

    table.to_csv('./' + out_file + '.csv',index=False)
    
    return table

In [77]:
devpath = './Composer_Dataset/NN_midi_files_extended/dev/'
testpath = './Composer_Dataset/NN_midi_files_extended/test/'
trainpath = './Composer_Dataset/NN_midi_files_extended/train/'

dev_table = create_files_table(devpath, 'dev_table')
test_table = create_files_table(testpath, 'test_table')
train_table = create_files_table(trainpath, 'train_table')

In [74]:
dev_table.head()

,Composers,Songs,Paths
0,bach,bach344,./Composer_Dataset/NN_midi_files_extended/dev/...
1,bach,bach348,./Composer_Dataset/NN_midi_files_extended/dev/...
2,bach,bach354,./Composer_Dataset/NN_midi_files_extended/dev/...
3,bach,bach384,./Composer_Dataset/NN_midi_files_extended/dev/...
4,bartok,bartok410,./Composer_Dataset/NN_midi_files_extended/dev/...
